# ENGR-E 533 Deep Learning Systems - Assignment 2

## Name : Akhila Sakiramolla (asakiram@iu.edu)
## UID : 2000886005

## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import librosa
import IPython.display as ipd
import soundfile as sf
import tensorflow as tf
import keras
from tensorflow.keras import initializers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers.core import Dense, Activation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Input

# To execute a cell line by line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Problem 1: Speech Denoising Using Deep Learning

In [2]:
# Loading data 

s, sr=librosa.load('train_clean_male.wav', sr=None)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)

In [3]:
# Creating train and validation split for clean signal

s_train = s[0:1007119]
s_val = s[-251780:]

In [4]:
# Creating train and validation split for noisy signal

sn_train = sn[0:1007119]
sn_val = sn[-251780:]

In [5]:
# Coverting clean signals into spectograms

S_train = librosa.stft(s_train, n_fft=1024, hop_length=512)
S_val = librosa.stft(s_val, n_fft=1024, hop_length=512)

S_train_abs  = np.transpose(np.abs(S_train))
S_train_abs.shape

S_val_abs  = np.transpose(np.abs(S_val))
S_val_abs.shape

(1968, 513)

(492, 513)

In [6]:
# Coverting noisy signals into spectograms

X_train = librosa.stft(sn_train, n_fft=1024, hop_length=512)
X_val = librosa.stft(sn_val, n_fft=1024, hop_length=512)

X_train_abs  = np.transpose(np.abs(X_train))
X_train_abs.shape

X_val_abs  = np.transpose(np.abs(X_val))
X_val_abs.shape

(1968, 513)

(492, 513)

In [7]:
# Building a model and adding 3 hidden layers with 512 hidden units

model = Sequential()
model.add(Dense(512, input_dim = 513, activation='relu',kernel_initializer=tf.keras.initializers.HeNormal()))
model.add(Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()))
model.add(Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()))
model.add(Dense(513, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()))


In [8]:
# Calculating loss and optimizing it

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

In [9]:
# Fitting the model on train dataset

model.fit(X_train_abs, S_train_abs, epochs=100, batch_size=128, shuffle=True)

Epoch 1/100
16/16 [==============================] - 1s 3ms/step - loss: 0.0850 - accuracy: 0.0523
Epoch 2/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0410 - accuracy: 0.1657
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0253 - accuracy: 0.2642
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0196 - accuracy: 0.3171
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0164 - accuracy: 0.3664
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0141 - accuracy: 0.3816
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0127 - accuracy: 0.3913
Epoch 8/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0120 - accuracy: 0.3958
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0109 - accuracy: 0.4085
Epoch 10/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0101 - accuracy: 0.4126
Epoch 11/

In [10]:
# Predicting on validation data 

S_test = model.predict(X_val_abs)

16/16 [==============================] - 0s 2ms/step


In [11]:
# Recovering the signal from predicted output

S_hat = np.multiply(X_val/X_val_abs.T,S_test.T)
S_rec = librosa.istft(S_hat, hop_length=512)
ipd.Audio(S_rec, rate = sr)

In [12]:
# Clipping clean signal to calculate SNR

s_val = abs(s_val[0:251392])
s_val.shape

(251392,)

In [13]:
# Calculating SNR value

SNR = 10*np.log10((np.sum(np.square(abs(s_val))))/(np.sum(np.square(abs(s_val) - abs(S_rec)))))
print("The SNR value for recovered signal : ", round(SNR,2))

The SNR value for recovered signal :  10.45


In [14]:
# Loading first test data and applying STFT

t1, sr = librosa.load('test_x_01.wav', sr=None)
T1 = librosa.stft(t1, n_fft=1024, hop_length=512)
T1_abs = np.abs(T1)
T1_abs = T1_abs.T
T1_abs.shape

(142, 513)

In [15]:
# Predicting on test data 
S_test1 = model.predict(T1_abs)

# Recovering the signal from predicted output
S_hat1 = np.multiply(T1/T1_abs.T,S_test1.T)
S_rec1 = librosa.istft(S_hat1, hop_length=512)
ipd.Audio(S_rec1, rate = sr)
sf.write('test_s_01_recons.wav', S_rec1, sr)

5/5 [==============================] - 0s 2ms/step


In [16]:
# Loading second test data and applying STFT

t2, sr = librosa.load('test_x_02.wav', sr=None)
T2 = librosa.stft(t2, n_fft=1024, hop_length=512)
T2_abs = np.abs(T2)
T2_abs = T2_abs.T
T2_abs.shape

(380, 513)

In [17]:
# Predicting on test data 
S_test2 = model.predict(T2_abs)

# Recovering the signal from predicted output
S_hat2 = np.multiply(T2/T2_abs.T,S_test2.T)
S_rec2 = librosa.istft(S_hat2, hop_length=512)
ipd.Audio(S_rec2, rate = sr)
sf.write('test_s_02_recons.wav', S_rec2, sr)

12/12 [==============================] - 0s 1ms/step


## Problem 2: Speech Denoising Using 1D CNN

In [18]:
# Loading data 

s, sr=librosa.load('train_clean_male.wav', sr=None)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)

In [19]:
# Creating train and validation split for clean signal

s_train = s[0:1007119]
s_val = s[-251780:]

In [20]:
# Creating train and validation split for noisy signal

sn_train = sn[0:1007119]
sn_val = sn[-251780:]

In [21]:
# Coverting clean signals into spectograms

S_train = librosa.stft(s_train, n_fft=1024, hop_length=512)
S_val = librosa.stft(s_val, n_fft=1024, hop_length=512)

S_train_abs  = np.transpose(np.abs(S_train))
S_train_abs.shape

S_val_abs  = np.transpose(np.abs(S_val))
S_val_abs.shape

(1968, 513)

(492, 513)

In [22]:
# Coverting noisy signals into spectograms

X_train = librosa.stft(sn_train, n_fft=1024, hop_length=512)
X_val = librosa.stft(sn_val, n_fft=1024, hop_length=512)

X_train_abs  = np.transpose(np.abs(X_train))
X_train_abs.shape

X_val_abs  = np.transpose(np.abs(X_val))
X_val_abs.shape

(1968, 513)

(492, 513)

In [23]:
# Building a model with 2 1D conloutional layers, 2 max pooling layers and 1 dense layer

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu',input_shape=(513,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(513, activation='relu'))

In [24]:
# Calculating loss and optimizing it

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

In [25]:
# Fitting the model

model.fit(X_train_abs, S_train_abs, epochs=100, batch_size=128, shuffle=True)

Epoch 1/100
16/16 [==============================] - 3s 41ms/step - loss: 0.0714 - accuracy: 0.0412
Epoch 2/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0430 - accuracy: 0.1550
Epoch 3/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0252 - accuracy: 0.2597
Epoch 4/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0169 - accuracy: 0.3283
Epoch 5/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0134 - accuracy: 0.3720
Epoch 6/100
16/16 [==============================] - 0s 7ms/step - loss: 0.0111 - accuracy: 0.3968
Epoch 7/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0097 - accuracy: 0.4060
Epoch 8/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0087 - accuracy: 0.4192
Epoch 9/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0084 - accuracy: 0.4309
Epoch 10/100
16/16 [==============================] - 0s 8ms/step - loss: 0.0079 - accuracy: 0.4360
Epoch 11

In [26]:
# Predicting on validation data 

S_test = model.predict(X_val_abs)

16/16 [==============================] - 0s 9ms/step


In [27]:
# Recovering the signal from predicted output

S_hat = np.multiply(X_val/X_val_abs.T,S_test.T)
S_rec = librosa.istft(S_hat, hop_length=512)
ipd.Audio(S_rec, rate = sr)

In [28]:
S_rec.shape

(251392,)

In [29]:
# Clipping clean signal to calculate SNR

s_val = abs(s_val[0:251392])
s_val.shape

(251392,)

In [30]:
# Calculating SNR value

SNR = 10*np.log10((np.sum(np.square(abs(s_val))))/(np.sum(np.square(abs(s_val) - abs(S_rec)))))
print("The SNR value for recovered signal : ", round(SNR,2))

The SNR value for recovered signal :  12.34


In [31]:
# Loading first test data and applying STFT

t1, sr = librosa.load('test_x_01.wav', sr=None)
T1 = librosa.stft(t1, n_fft=1024, hop_length=512)
T1_abs = np.abs(T1)
T1_abs = T1_abs.T
T1_abs.shape

(142, 513)

In [32]:
# Predicting on test data 

S_test1 = model.predict(T1_abs)

5/5 [==============================] - 0s 31ms/step


In [33]:
# Recovering the signal from predicted output

S_hat1 = np.multiply(T1/T1_abs.T,S_test1.T)
S_rec1 = librosa.istft(S_hat1, hop_length=512)
ipd.Audio(S_rec1, rate = sr)
sf.write('test_s_01_recons_1Dcnn.wav', S_rec1, sr)

In [34]:
# Loading second test data and applying STFT

t2, sr = librosa.load('test_x_02.wav', sr=None)
T2 = librosa.stft(t2, n_fft=1024, hop_length=512)
T2_abs = np.abs(T2)
T2_abs = T2_abs.T
T2_abs.shape

(380, 513)

In [35]:
# Predicting on test data 

S_test2 = model.predict(T2_abs)

12/12 [==============================] - 0s 15ms/step


In [36]:
# Recovering the signal from predicted output

S_hat2 = np.multiply(T2/T2_abs.T,S_test2.T)
S_rec2 = librosa.istft(S_hat2, hop_length=512)
ipd.Audio(S_rec2, rate = sr)
sf.write('test_s_02_recons_1Dcnn.wav', S_rec2, sr)

## Problem 3: Speech Denoising Using 2D CNN

In [158]:
# Loading data 

s, sr=librosa.load('train_clean_male.wav', sr=None)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)

In [159]:
# Creating train and validation split for clean signal

s_train = s[0:1007119]
s_val = s[-251780:]

In [160]:
# Creating train and validation split for noisy signal

sn_train = sn[0:1007119]
sn_val = sn[-251780:]

In [161]:
# Coverting clean signals into spectograms

S_train = librosa.stft(s_train, n_fft=1024, hop_length=512)
S_val = librosa.stft(s_val, n_fft=1024, hop_length=512)

S_train_abs  = np.transpose(np.abs(S_train))
S_train_abs.shape

S_val_abs  = np.transpose(np.abs(S_val))
S_val_abs.shape

(1968, 513)

(492, 513)

In [162]:
# Coverting noisy signals into spectograms

X_train = librosa.stft(sn_train, n_fft=1024, hop_length=512)
X_val = librosa.stft(sn_val, n_fft=1024, hop_length=512)

X_train_abs  = np.transpose(np.abs(X_train))
X_train_abs.shape

X_val_abs  = np.transpose(np.abs(X_val))
X_val_abs.shape

(1968, 513)

(492, 513)

In [163]:
w = 20
train_data = X_train_abs[:w,:]

h = 1
while h <= X_train_abs.shape[0]-w:
  train_data = np.append(train_data,X_train_abs[h:h+w,:],axis=0)
  h += 1

x_train = train_data.reshape((1949,20,513,1))
x_train.shape

y_train = S_train_abs[:1949,:]
y_train = y_train.reshape((1949,513,1))
y_train.shape

(1949, 20, 513, 1)

(1949, 513, 1)

In [164]:
w = 20
val_data = X_val_abs[:w,:]

h = 1
while h <= X_val_abs.shape[0]-w:
  val_data = np.append(val_data,X_val_abs[h:h+w,:],axis=0)
  h += 1

x_val = val_data.reshape((473,20,513,1))
x_val.shape

(473, 20, 513, 1)

In [165]:
# Building a model with 2 22D conloutional layers and 2 max pooling layers 

model = Sequential()
model.add(Input(shape = (20,513,1)))
model.add(Conv2D(100, kernel_size=(7,7), activation='relu', strides = 1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
model.add(Conv2D(50, kernel_size=(5,5), activation='relu',strides = 1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
model.add(Flatten())
model.add(Dense(513, activation='relu',kernel_initializer = tf.keras.initializers.HeNormal()))

In [166]:
# Calculating loss and optimizing it

model.compile(optimizer='adam',loss='mse')

In [167]:
# Fitting the model

model.fit(x=x_train, y=y_train, batch_size=100, epochs=130, shuffle=True)

Epoch 1/130
20/20 [==============================] - 2s 81ms/step - loss: 0.0823
Epoch 2/130
20/20 [==============================] - 1s 53ms/step - loss: 0.0665
Epoch 3/130
20/20 [==============================] - 1s 53ms/step - loss: 0.0572
Epoch 4/130
20/20 [==============================] - 1s 52ms/step - loss: 0.0477
Epoch 5/130
20/20 [==============================] - 1s 52ms/step - loss: 0.0423
Epoch 6/130
20/20 [==============================] - 1s 52ms/step - loss: 0.0355
Epoch 7/130
20/20 [==============================] - 1s 52ms/step - loss: 0.0298
Epoch 8/130
20/20 [==============================] - 1s 53ms/step - loss: 0.0255
Epoch 9/130
20/20 [==============================] - 1s 53ms/step - loss: 0.0233
Epoch 10/130
20/20 [==============================] - 1s 53ms/step - loss: 0.0211
Epoch 11/130
20/20 [==============================] - 1s 53ms/step - loss: 0.0194
Epoch 12/130
20/20 [==============================] - 1s 53ms/step - loss: 0.0174
Epoch 13/130
20/20 [=====

In [168]:
# Predicting on validation data 

S_test = model.predict(x_val)

15/15 [==============================] - 0s 6ms/step


In [169]:
X_val1 = X_val[:,:473]
X_val_abs1 = X_val_abs[:473,:]

In [170]:
# Recovering the signal from predicted output

S_hat = np.multiply(X_val1/X_val_abs1.T,S_test.T)
S_rec = librosa.istft(S_hat, hop_length=512)
ipd.Audio(S_rec, rate = sr)

In [171]:
S_rec.shape

(241664,)

In [172]:
# Clipping clean signal to calculate SNR

s_val = abs(s_val[0:241664])
s_val.shape

(241664,)

In [173]:
# Calculating SNR value

SNR = 10*np.log10((np.sum(np.square(abs(s_val))))/(np.sum(np.square(abs(s_val) - abs(S_rec)))))
print("The SNR value for recovered signal : ", round(SNR,2))

The SNR value for recovered signal :  7.46


In [174]:
# Loading first test data and applying STFT

t1, sr = librosa.load('test_x_01.wav', sr=None)
T1 = librosa.stft(t1, n_fft=1024, hop_length=512)
T1_abs = np.abs(T1)
T1_abs = T1_abs.T
T1_abs.shape

(142, 513)

In [175]:
w = 20
train_data = T1_abs[:w,:]

h = 1
while h <= T1_abs.shape[0]-w:
  train_data = np.append(train_data,T1_abs[h:h+w,:],axis=0)
  h += 1

x_train = train_data.reshape((123,20,513,1))
x_train.shape

(123, 20, 513, 1)

In [176]:
# Predicting the model output

pred = model.predict(x_train,batch_size=32,verbose=0)

In [177]:
# Recovering the signal from predicted output

T1 = T1[:,:123]
T1_mag = (T1/np.abs(T1))
T1_rec = T1_mag*pred.T
T1_rec = librosa.istft(T1_rec,hop_length=512)
ipd.Audio(T1_rec, rate=sr)
sf.write('test_s_01_recons_2Dcnn.wav', T1_rec, sr)

In [178]:
# Loading second test data and applying STFT

t2, sr = librosa.load('test_x_02.wav', sr=None)
T2 = librosa.stft(t2, n_fft=1024, hop_length=512)
T2_abs = np.abs(T2)
T2_abs = T2_abs.T
T2_abs.shape

(380, 513)

In [179]:
w = 20
train_data = T2_abs[:w,:]

h = 1
while h <= T2_abs.shape[0]-w:
  train_data = np.append(train_data,T2_abs[h:h+w,:],axis=0)
  h += 1

x_train = train_data.reshape((361,20,513,1))
x_train.shape

(361, 20, 513, 1)

In [180]:
# Predicting the model output

pred = model.predict(x_train,batch_size=32,verbose=0)

In [181]:
# Recovering the signal from predicted output

T2 = T2[:,:361]
T2_mag = (T2/np.abs(T2))
T2_rec = T2_mag*pred.T
T2_rec = librosa.istft(T2_rec,hop_length=512)
ipd.Audio(T2_rec, rate=sr)
sf.write('test_s_02_recons_2Dcnn.wav', T2_rec, sr)